In [92]:
import pandas as pd
import seaborn as sns
import numpy as np
import os.path as op
import glob
import matplotlib.pyplot as plt
import ast
# import moss
import csv
import random
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import os
from pandas import Timestamp
from datetime import timedelta


In [93]:
home_dir = op.abspath('./')

data_files = glob.glob(op.join(home_dir,
                        'data',
                        '*.csv'))
sns.set_context('talk')

In [94]:
column_name = 'image_recall_response.keys'
threshold = 0.4

In [95]:
creation_dates = []
for file_path in data_files:
    try:
        df = pd.read_csv(file_path)
        if column_name in df.columns:
            creation_date = df['date']
            creation_dates.append(creation_date)
    except Exception as e:
#             print(f"Error reading {file_path}: {e}")
        continue

In [96]:
def parse_dates(series):
    date_str = series.iloc[0]
    if "24h" in date_str:
        corrected_date_str = date_str.replace("24h", "00h")
        dt = pd.to_datetime(corrected_date_str, format='%Y-%m-%d_%Hh%M.%S.%f')
        dt += timedelta(days=1)
    else:
        dt = pd.to_datetime(date_str, format='%Y-%m-%d_%Hh%M.%S.%f')
    return dt

dates = [parse_dates(series) for series in creation_dates]

min_date = min(dates)


In [97]:
def remove_unit_variance(df, col, unit, group=None, suffix="_within"):
    """Remove variance between sampling units.

    This is useful for plotting repeated-measures data using within-unit
    error bars.

    Parameters
    ----------
    df : DataFrame
        Input data. Will have a new column added.
    col : column name
        Column in dataframe with quantitative measure to modify.
    unit : column name
        Column in dataframe defining sampling units (e.g., subjects).
    group : column name(s), optional
        Columns defining groups to remove unit variance within.
    suffix : string, optional
        Suffix appended to ``col`` name to create new column.

    Returns
    -------
    df : DataFrame
        Returns modified dataframe.

    """
    new_col = col + suffix

    def demean(x):
        return x - x.mean()

    if group is None:
        new = df.groupby(unit)[col].transform(demean)
        new += df[col].mean()
        df.loc[:, new_col] = new
    else:
        df.loc[:, new_col] = np.nan
        for level, df_level in df.groupby(group):
            new = df_level.groupby(unit)[col].transform(demean)
            new += df_level[col].mean()
            df.loc[new.index, new_col] = new

    return df


In [98]:
# df['resp_correct']

In [99]:
def convert_string(value):
    try:
        # Check if value is a string and looks like a list
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            # Remove brackets and split by comma
            cleaned = value.strip('[]')
            if cleaned:  # Check if the string is not empty
                # Convert each part to float and calculate average
                numbers = [float(num) for num in cleaned.split(',')]
                return sum(numbers) / len(numbers)
            else:
                return None  # Return None or another placeholder for empty lists
        else:
            return None
    except Exception as e:
        print(f"Error converting value {value}: {e}")
        return None


In [100]:
def categorize_columns(dataframe, column_params):
    """
    Categorizes specified columns in a DataFrame into discrete categories based on quantiles.

    Parameters:
    - dataframe: The DataFrame to process.
    - column_params: A dictionary specifying the number of categories and labels for each column prefix.

    Returns:
    - The modified DataFrame with additional columns for categorized data.
    """
    for label in ['it_sim_dis_attend', 'v2_sim_dis_attend', 'it_sim_dis_unattend', 'v2_sim_dis_unattend', 'it_sim_dis_diff', 'v2_sim_dis_diff', 'it_im1_im2', 'v2_im1_im2', 'it_sim_dis_test', 'v2_sim_dis_test', 'it_sim_dis_untest', 'v2_sim_dis_untest', 'it_sim_dis_diff_test', 'v2_sim_dis_diff_test']:
        # Determine the column prefix to decide which set of parameters to use
        column_prefix = 'v2' if 'v2' in label else 'it'
        
        # Extract the number of categories and labels from the parameters dictionary
        n_cats = column_params[column_prefix]['n_cats']
        labels = column_params[column_prefix]['labels']
        
        # Categorize the column data
        dataframe[label + '_cat'] = pd.qcut(dataframe[label], q=n_cats, labels=labels, duplicates='drop')
    
    return dataframe

In [101]:
def validity_assignment(df):
    x = []
    for i in df['validity']:
        if i == 'valid':
            x.append('prioritized')
        else:
            x.append('deprioritized')

    df['Tested Item'] = x

In [102]:
def calculate_differences(dataframe):
    # Determine attended and unattended values for IT and V2
    dataframe['it_sim_dis_attend'] = np.where(dataframe['attend'] == 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_attend'] = np.where(dataframe['attend'] == 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    dataframe['it_sim_dis_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    
    dataframe['it_sim_dis_unattend'] = np.where(dataframe['attend'] != 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_unattend'] = np.where(dataframe['attend'] != 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    
    dataframe['it_sim_dis_untest'] = np.where(dataframe['test_item'] != 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_untest'] = np.where(dataframe['test_item'] != 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    # Calculate differences
    dataframe['it_sim_dis_diff'] = np.where(dataframe['attend'] == 'img1', dataframe['IT_root_im1'] - dataframe['IT_root_im2'], dataframe['IT_root_im2'] - dataframe['IT_root_im1'])
    dataframe['v2_sim_dis_diff'] = np.where(dataframe['attend'] == 'img1', dataframe['V2_root_im1'] - dataframe['V2_root_im2'], dataframe['V2_root_im2'] - dataframe['V2_root_im1'])

    dataframe['it_sim_dis_diff_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['IT_root_im1'] - dataframe['IT_root_im2'], dataframe['IT_root_im2'] - dataframe['IT_root_im1'])
    dataframe['v2_sim_dis_diff_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['V2_root_im1'] - dataframe['V2_root_im2'], dataframe['V2_root_im2'] - dataframe['V2_root_im1'])

    # Determine convergence and preferences
    dataframe['v2_converges'] = np.where((dataframe['it_sim_dis_diff'] > 0) & (dataframe['v2_sim_dis_diff'] > 0) | (dataframe['it_sim_dis_diff'] < 0) & (dataframe['v2_sim_dis_diff'] < 0), 'V2/IT agree', 'V2/IT disagree')

    dataframe['v2_prefers'] = np.where(dataframe['v2_sim_dis_diff'] > 0, 'Prioritized', 'Deprioritized')
    dataframe['it_prefers'] = np.where(dataframe['it_sim_dis_diff'] > 0, 'Prioritized', 'Deprioritized')
    
    
    dataframe['v2_prefers_test'] = np.where(dataframe['v2_sim_dis_diff_test'] > 0, 'Tested', 'Untested')
    dataframe['it_prefers_test'] = np.where(dataframe['it_sim_dis_diff_test'] > 0, 'Tested', 'Untested')
    
    
    dataframe['Distractor V2 Similarity Preference Tested'] = dataframe['v2_prefers_test'] 
    dataframe['Distractor IT Similarity Preference Tested'] = dataframe['it_prefers_test']
    
    
    
    dataframe['Distractor V2 Similarity Preference'] = dataframe['v2_prefers'] 
    dataframe['Distractor IT Similarity Preference'] = dataframe['it_prefers']
    
    dataframe['it_im1_im2'] = dataframe['IT_im1_im2']
    dataframe['v2_im1_im2'] = dataframe['V2_im1_im2']
    
    dataframe['IT_diff_binned'] = pd.qcut(dataframe['it_sim_dis_diff'], 5, duplicates='drop')
    dataframe['V2_diff_binned'] = pd.qcut(dataframe['v2_sim_dis_diff'], 5, duplicates='drop')
    
    dataframe['IT_diff_binned_test'] = pd.qcut(dataframe['it_sim_dis_diff_test'], 5, duplicates='drop')
    dataframe['V2_diff_binned_test'] = pd.qcut(dataframe['v2_sim_dis_diff_test'], 5, duplicates='drop')
#     dataframe['IT_interim_binned'] = pd.qcut(dataframe['it_sim_dis_diff'], 5, duplicates='drop')
#     dataframe['V2_interim_binned'] = pd.qcut(dataframe['v2_sim_dis_diff'], 5, duplicates='drop')
    return dataframe

In [103]:
def df_column_addition(df):
    df['V2 Distractor Similarity\nto Prioritized Item'] = df['v2_sim_dis_attend_cat']
    df['IT Distractor Similarity\nto Prioritized Item'] = df['it_sim_dis_attend_cat']
    
    df['V2 Distractor Similarity\nto Tested Item'] = df['v2_sim_dis_test_cat']
    df['IT Distractor Similarity\nto Tested Item'] = df['it_sim_dis_test_cat']
    
    df['V2 Distractor Similarity\nto Deprioritized Item'] = df['v2_sim_dis_unattend_cat']
    df['IT Distractor Similarity\nto Deprioritized Item'] = df['it_sim_dis_unattend_cat']
    
    df['V2 Distractor Similarity\nto Untested Item'] = df['v2_sim_dis_untest_cat']
    df['IT Distractor Similarity\nto Untested Item'] = df['it_sim_dis_untest_cat']
    
    df['Tested - Untested IT Distractor Similarity'] = df['it_sim_dis_diff_test_cat'] 
    df['Tested - Untested V2 Distractor Similarity'] = df['v2_sim_dis_diff_test_cat'] 
    
    df['Prioritized - Deprioritized IT Distractor Similarity'] = df['it_sim_dis_diff_cat'] 
    df['Prioritized - Deprioritized V2 Distractor Similarity'] = df['v2_sim_dis_diff_cat'] 
    
    
    
    df['it_sim_dis_diff_sq'] = df['it_sim_dis_diff'] ** 2
    df['v2_sim_dis_diff_sq'] = df['v2_sim_dis_diff'] ** 2
    df['v2_sim_dis_attend_sq'] = df['v2_sim_dis_attend'] ** 2
    df['it_sim_dis_attend_sq'] = df['it_sim_dis_attend']**2
    df['it_sim_dis_unattend_sq'] = df['it_sim_dis_unattend']**2
    df['v2_sim_dis_unattend_sq'] = df['v2_sim_dis_unattend']**2
    
    df['it_sim_dis_diff_test_sq'] = df['it_sim_dis_diff_test'] ** 2
    df['v2_sim_dis_diff_test_sq'] = df['v2_sim_dis_diff_test'] ** 2
    df['v2_sim_dis_test_sq'] = df['v2_sim_dis_test'] ** 2
    df['it_sim_dis_test_sq'] = df['it_sim_dis_test']**2
    df['it_sim_dis_untest_sq'] = df['it_sim_dis_untest']**2
    df['v2_sim_dis_untest_sq'] = df['v2_sim_dis_untest']**2

In [104]:
def df_creation(data_files, start_date, end_date):
    processed_dfs = []
    date_column = 'date'

    for file_path in data_files:
        try:
            # Read the CSV file and append to the list of DataFrames
            temp_df = pd.read_csv(file_path)
            temp_df['filename'] = file_path
            processed_dfs.append(temp_df)
        except Exception as e:
            # Optionally, print or log the error
#             print(f"Failed to read {file_path}: {e}")
            continue

    if processed_dfs:
        df = pd.concat(processed_dfs, ignore_index=True)
        df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d_%Hh%M.%S.%f', errors='coerce')
        df.dropna(subset=[date_column], inplace=True)
        df = df[(df[date_column] >= pd.to_datetime(start_date)) & (df[date_column] <= pd.to_datetime(end_date))]
        
        # # Uncomment and adapt the following lines as needed:
        df = df.loc[df['V2_diff'].notnull()].reset_index(drop=True)
        df['reliability'] = df['reliability'].astype(float)
        df['Retrocue Reliability'] = np.where(df['reliability'] > 0.75, 'high', 'low')
        non_numeric_values = df['resp_correct'][~df['resp_correct'].apply(np.isreal)]
        numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        result = df[numeric_columns].groupby('participant').mean()
        
        

    else:
        # Return an empty DataFrame if no files were processed successfully
        df = pd.DataFrame()
    
    return df, numeric_columns


   

In [105]:
# df['resp_correct']

In [106]:
df, numeric_columns = df_creation(data_files,  '2024-11-22', '2025-01-30')
# '2024-10-08', '2024-10-30'
###'2024-11-22', '2025-01-30'

In [107]:
def add_trial_info(df, participant_col, trials_per_batch=30):
#     result = df.copy()
    
    df['Trial_Number'] = df.groupby(participant_col).cumcount() + 1
    
    df['Trial_Batch'] = ((df['Trial_Number'] - 1) // trials_per_batch) + 1
    
    return df

In [108]:
add_trial_info(df, participant_col='participant')  

,setup_js.started,setup_js.stopped,participant,date,expName,psychopyVersion,OS,frameRate,Instructions1.started,Instructions1.stopped,...,trials_3.thisIndex,trials_3.ran,trials_3.order,taskPhases,conditions_choice,repeat_prac.order,prac_trials.order,Retrocue Reliability,Trial_Number,Trial_Batch
0,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,low,1,1
1,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,2,1
2,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,3,1
3,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,4,1
4,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,low,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51801,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,296,10
51802,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,297,10
51803,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,low,298,10
51804,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,299,10


In [109]:
# df['Trial_Batch']

In [110]:
# df_with_conditions_batch = add_conditions_batch(df)


In [111]:
# df_with_conditions_batch['conditions_batch']

In [112]:
df_trial_182 = df[df['trial'] == 182]
df_trial_182 = df_trial_182[['participant', 'trial', 'cond_file', 'root', 'IT_diff']]
df_trial_182['conditions_batch'] = df_trial_182.groupby(['cond_file', 'root', 'IT_diff']).ngroup() + 1

batch_info = df_trial_182.groupby('conditions_batch')['participant'].agg(
    participants_count='nunique',  # Count of unique participants
    participants_list='unique'  # List of participants in the batch
).reset_index()

df_trial_182 = df_trial_182.merge(batch_info, on='conditions_batch', how='left')


#pd.set_option('display.max_rows', None)  # This will show all rows
#pd.set_option('display.max_columns', None)  # This will show all columns
#pd.set_option('display.width', None)  # Prevent truncation of wide output
#pd.set_option('display.max_colwidth', None)  # Prevent truncation of column content
# print(df_trial_182)

In [113]:
# df_trial_182

In [114]:
df = df.merge(df_trial_182[['participant', 'conditions_batch', 'participants_count', 'participants_list']], 
                     on='participant', 
                     how='left')

# Display the resulting dataframe
# print(df_merged)

In [115]:
df['participants_count'].max()

7.0

In [116]:
# df = df_merged

In [117]:
df.to_csv('pilot6_unprocessed.csv', index=False)


In [118]:
unique_task_phases = df['taskPhase'].unique()
unique_task_phases

array(['mainTask'], dtype=object)

In [119]:
df_XXX = df[df['image_recall_response.keys'].notna()]
# df_XXX

In [120]:
len(set(df['participant']))

173

In [121]:
# df['resp_correct']

In [122]:
def process_values_time(value):
    try:
        # Strip square brackets and split by comma, filtering out empty values
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            cleaned = value.strip('[]').split(',')
            if cleaned == ['']:  # Check if the list after stripping is empty
                return None
            numbers = [float(num) for num in cleaned if num.strip()]
            return numbers
        return None
    except ValueError:
        return None

def process_values_click(value):
    try:
        # Strip square brackets and split by comma, filtering out empty values
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            cleaned = value.strip('[]').split(',')
            if cleaned == ['']:  # Check if the list after stripping is empty
                return None
            numbers = [num for num in cleaned if num.strip()]
            return numbers
        return None
    except ValueError:
        return None


In [123]:
df['processed_mouse.time'] = df['mouse.time'].apply(process_values_time)
df['processed_mouse.click'] = df['mouse.clicked_name'].apply(process_values_click)



In [124]:
# df['processed_mouse.click']

In [125]:
def get_latest_value(entry):
    if isinstance(entry, list) and len(entry) > 1:
        return [entry[-1]]  # Return a list with the latest value
    return entry  # Return unchanged if not a list or list is empty/single element

df['processed_mouse.time'] = df['processed_mouse.time'].apply(get_latest_value)

print("Updated DataFrame with latest values in 'processed_mouse.time':")
# print(df)

Updated DataFrame with latest values in 'processed_mouse.time':


In [126]:
participant_summary = df.groupby('participant').agg(
    total_entries=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()

# print(participant_summary)

In [127]:
participants_over_300 = participant_summary[participant_summary['total_entries'] > 300]

# Display the summary of total and removed entries per participant
print("Participant Summary:")
# print(participant_summary)

print("\nParticipants with more than 300 entries:")
# print(participants_over_300)


Participant Summary:

Participants with more than 300 entries:


In [128]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Find the earliest date for each participant
earliest_dates = df.groupby('participant')['date'].min().reset_index()

# Merge with the original dataframe to filter the rows with the earliest date for each participant
df = pd.merge(df, earliest_dates, on=['participant', 'date'])

# Display the cleaned DataFrame
# df


In [129]:
participant_file_counts = df.groupby(['participant', 'filename']).agg(
    entry_count=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()
duplicate_participants = participant_file_counts[participant_file_counts.duplicated(subset=['participant'], keep=False)]

# Display the result
print(duplicate_participants)

Empty DataFrame
Columns: [participant, filename, entry_count]
Index: []


In [130]:
participant_summary = df.groupby('participant').agg(
    total_entries=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()

participant_file_counts = df.groupby(['participant', 'filename']).agg(
    entry_count=pd.NamedAgg(column='participant', aggfunc='size'),
    
    none_mouse=pd.NamedAgg(column='processed_mouse.time', aggfunc=lambda x: x.isna().sum())
).reset_index()


files_with_high_none_mouse = participant_file_counts[participant_file_counts['none_mouse'] > 100]

df_cleaned = df[~df.set_index(['participant', 'filename']).index.isin(files_with_high_none_mouse.set_index(['participant', 'filename']).index)]

participant_file_counts_cleaned = df_cleaned.groupby(['participant', 'filename']).agg(
    entry_count=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()

files_with_low_entries = participant_file_counts_cleaned[participant_file_counts_cleaned['entry_count'] < 200]

df_cleaned = df_cleaned[~df_cleaned.set_index(['participant', 'filename']).index.isin(files_with_low_entries.set_index(['participant', 'filename']).index)]

cleaned_participant_summary = df_cleaned.groupby('participant').agg(
    total_entries=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()


none_mouse_summary = df_cleaned.groupby('participant').agg(
    none_mouse=pd.NamedAgg(column='processed_mouse.time', aggfunc=lambda x: x.isna().sum())
).reset_index()

print("\nNone mouse summary for each participant:")
# print(none_mouse_summary)



None mouse summary for each participant:


In [131]:
df = df_cleaned
# df

In [132]:
duplicate_participants = df.groupby('participant')['filename'].nunique().reset_index()
duplicate_participants = duplicate_participants[duplicate_participants['filename'] > 1]

if not duplicate_participants.empty:
    print("\nParticipants associated with more than one file:")
    print(duplicate_participants)
else:
    print("\nNo participants associated with more than one file.")


No participants associated with more than one file.


In [133]:
none_mouse_summary = df.groupby('participant').agg(
    none_mouse=pd.NamedAgg(column='processed_mouse.time', aggfunc=lambda x: x.isna().sum())
).reset_index()

# Check for participants with none_mouse more than 2/3 of total_entries
none_mouse_summary = none_mouse_summary.merge(cleaned_participant_summary, on='participant')
none_mouse_summary['ratio'] = none_mouse_summary['none_mouse'] / none_mouse_summary['total_entries']
participants_high_none_mouse = none_mouse_summary[none_mouse_summary['ratio'] > (2/3)]

# Print the results
print("Remaining participants with more than 300 entries:")
# print(cleaned_participant_summary)

print("\nNone mouse summary for each participant:")
# print(none_mouse_summary)

if not participants_high_none_mouse.empty:
    print("\nParticipants with none_mouse more than 2/3 of total entries:")
#     print(participants_high_none_mouse)
else:
    print("\nNo participants have none_mouse more than 2/3 of total entries.")

Remaining participants with more than 300 entries:

None mouse summary for each participant:

No participants have none_mouse more than 2/3 of total entries.


In [134]:
participant_counts = df['participant'].value_counts()
participants_less_than_300 = participant_counts[participant_counts < 300]
# participant_counts


In [135]:
def add_length_column(df, column_name):
    if column_name in df.columns:
        # Clean and convert the values in the specified column
        def clean_and_get_length(value):
            if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
                cleaned = value.strip('[]').split(',')
                return len(cleaned)
            elif isinstance(value, list):
                return 0
            else:
                return 0  # Handle unexpected data types

        # Apply the cleaning and length calculation
        df.loc[:, column_name + '_length'] = df[column_name].apply(clean_and_get_length)
    else:
        print(f"Column '{column_name}' not found in DataFrame.")
    return df

def add_nthresp_column(df, column_name, n):
    if column_name in df.columns:
        # Clean and convert the values in the specified column
        def clean_and_get_nthresp(value):
            if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
                cleaned = value.strip('[]').split(',')
                return cleaned[n]
            elif isinstance(value, list):
                return 
            else:
                return  # Handle unexpected data types

        # Apply the cleaning and length calculation
        if n == 0:
            temp_str = '_first'
        elif n == -1:
            temp_str = '_last'
        df.loc[:, column_name + temp_str] = df[column_name].apply(clean_and_get_nthresp)
    else:
        print(f"Column '{column_name}' not found in DataFrame.")
    return df

In [136]:
add_length_column(df, 'mouse.time')


,setup_js.started,setup_js.stopped,participant,date,expName,psychopyVersion,OS,frameRate,Instructions1.started,Instructions1.stopped,...,prac_trials.order,Retrocue Reliability,Trial_Number,Trial_Batch,conditions_batch,participants_count,participants_list,processed_mouse.time,processed_mouse.click,mouse.time_length
0,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,low,1,1,42.0,3.0,"[165862, 131011, 155275]",[2.579600000023902],"[""memoranda_stim""]",1
1,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,high,2,1,42.0,3.0,"[165862, 131011, 155275]",[1.9633999999762182],"[""distractor_im_3""]",1
2,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,high,3,1,42.0,3.0,"[165862, 131011, 155275]",[2.3300999999046326],"[""memoranda_stim""]",1
3,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,high,4,1,42.0,3.0,"[165862, 131011, 155275]",[1.4297999999523654],"[""memoranda_stim""]",1
4,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,NaN,low,5,1,42.0,3.0,"[165862, 131011, 155275]",[2.579799999952229],"[""memoranda_stim""]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51801,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,high,296,10,43.0,1.0,[167431],[2.561999999999898],"[""memoranda_stim""]",2
51802,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,high,297,10,43.0,1.0,[167431],[2.030999999999949],"[""memoranda_stim""]",2
51803,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,low,298,10,43.0,1.0,[167431],[2.0809999999992215],"[""distractor_im_1""]",1
51804,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,NaN,high,299,10,43.0,1.0,[167431],[2.16399999999976],"[""memoranda_stim""]",2


In [137]:
add_nthresp_column(df, 'mouse.time', 0)
add_nthresp_column(df, 'mouse.time', -1)
add_nthresp_column(df, 'mouse.clicked_name', 0)
add_nthresp_column(df, 'mouse.clicked_name', -1)


,setup_js.started,setup_js.stopped,participant,date,expName,psychopyVersion,OS,frameRate,Instructions1.started,Instructions1.stopped,...,conditions_batch,participants_count,participants_list,processed_mouse.time,processed_mouse.click,mouse.time_length,mouse.time_first,mouse.time_last,mouse.clicked_name_first,mouse.clicked_name_last
0,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,42.0,3.0,"[165862, 131011, 155275]",[2.579600000023902],"[""memoranda_stim""]",1,2.579600000023902,2.579600000023902,"""memoranda_stim""","""memoranda_stim"""
1,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,42.0,3.0,"[165862, 131011, 155275]",[1.9633999999762182],"[""distractor_im_3""]",1,1.9633999999762182,1.9633999999762182,"""distractor_im_3""","""distractor_im_3"""
2,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,42.0,3.0,"[165862, 131011, 155275]",[2.3300999999046326],"[""memoranda_stim""]",1,2.3300999999046326,2.3300999999046326,"""memoranda_stim""","""memoranda_stim"""
3,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,42.0,3.0,"[165862, 131011, 155275]",[1.4297999999523654],"[""memoranda_stim""]",1,1.4297999999523654,1.4297999999523654,"""memoranda_stim""","""memoranda_stim"""
4,NaN,NaN,165862,2024-11-27 10:29:27.557,WM_Deepgen,2023.2.1,Win32,59.88024,NaN,NaN,...,42.0,3.0,"[165862, 131011, 155275]",[2.579799999952229],"[""memoranda_stim""]",1,2.579799999952229,2.579799999952229,"""memoranda_stim""","""memoranda_stim"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51801,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,43.0,1.0,[167431],[2.561999999999898],"[""memoranda_stim""]",2,1.5450000000000728,2.561999999999898,"""memoranda_stim""","""memoranda_stim"""
51802,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,43.0,1.0,[167431],[2.030999999999949],"[""memoranda_stim""]",2,1.0139999999992142,2.030999999999949,"""memoranda_stim""","""memoranda_stim"""
51803,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,43.0,1.0,[167431],[2.0809999999992215],"[""distractor_im_1""]",1,2.0809999999992215,2.0809999999992215,"""distractor_im_1""","""distractor_im_1"""
51804,NaN,NaN,167431,2024-11-24 14:54:08.781,WM_Deepgen,2023.2.1,MacIntel,62.50000,NaN,NaN,...,43.0,1.0,[167431],[2.16399999999976],"[""memoranda_stim""]",2,1.1299999999991996,2.16399999999976,"""memoranda_stim""","""memoranda_stim"""


In [138]:
long_instances = df[df['mouse.time_length'] > 1]
# print(long_instances)


In [139]:
unique_mouse_time_length_values = df['mouse.time_length'].unique()
unique_mouse_time_length_values

array([ 1,  2,  4,  3,  5,  7,  6,  8, 11, 10, 12,  9, 14, 15, 13, 17])

In [140]:
if 'participant' in df.columns and not df['participant'].isnull().all():
    print("'participant' column exists and has values.")
else:
    raise KeyError("The 'participant' column is missing or has no values.")

'participant' column exists and has values.


In [141]:
def clean_dataframe_from_nonresp(df):
    # Convert the column entries from string lists to actual lists of floats, or None for empty lists

    # Define a mask to filter out rows where the list is empty or any number is out of the specified range
    def filter_ranges(numbers):
        if numbers is None:
            return False
        return all(num for num in numbers)
    df = df[df['processed_mouse.time'].apply(filter_ranges)]
    df = df[df['processed_mouse.click'].apply(filter_ranges)]
    # Apply the mask
    return df

In [142]:
len(set(df['participant']))

150

In [143]:
if 'participant' in df.columns and not df['participant'].isnull().all():
    print("'participant' column exists and has values.")
else:
    raise KeyError("The 'participant' column is missing or has no values.")

'participant' column exists and has values.


In [144]:
def df_with_threshold(df, numeric_columns, threshold):
    sub_df = df[numeric_columns].groupby('participant').mean().reset_index()
    above04_subs = sub_df.loc[sub_df['resp_correct'] >= threshold, 'participant']
    df_ret = pd.DataFrame()
    for s in above04_subs:
        df_ret = pd.concat([df_ret, df.loc[df['participant'] == s]])
    df_ret = remove_unit_variance(df_ret,'resp_correct','participant')
    df_ret['Accuracy'] = df_ret['resp_correct_within']
    return df_ret

In [145]:
# df['resp_correct_within'].max()

In [146]:
df = df_with_threshold(df, numeric_columns, 0.4)

In [147]:
len(set(df['participant']))

137

In [148]:
column_params = {
    'v2': {'n_cats': 5, 'labels': ['Least Similar', '', ' ', '   ', 'Most Similar']},
    'it': {'n_cats': 5, 'labels': ['Least Similar', '', ' ', '   ', 'Most Similar']}
}
column_name = 'image_recall_response.keys'
threshold = 0.4

In [149]:
def interval_to_str(interval):
    return f"({interval.left:.2f}, {interval.right:.2f}]"
def label_addition(df):
    # Map the existing intervals to their new formatted string representations
    new_labels1 = df['IT_diff_binned'].cat.categories.map(interval_to_str)
    new_labels2= df['V2_diff_binned'].cat.categories.map(interval_to_str)

    new_labels3 = df['IT_diff_binned_test'].cat.categories.map(interval_to_str)
    new_labels4= df['V2_diff_binned_test'].cat.categories.map(interval_to_str)

    # Rename the categories of the existing column
    df['IT_diff_binned'] = df['IT_diff_binned'].cat.rename_categories(new_labels1)
    df['V2_diff_binned'] = df['V2_diff_binned'].cat.rename_categories(new_labels2)
    df['IT_diff_binned_test'] = df['IT_diff_binned_test'].cat.rename_categories(new_labels3)
    df['V2_diff_binned_test'] = df['V2_diff_binned_test'].cat.rename_categories(new_labels4)



In [150]:
validity_assignment(df)
df = calculate_differences(df)
df = categorize_columns(df, column_params)
df_column_addition(df)
label_addition(df)

/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_48599/2665625952.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe['it_prefers_test'] = np.where(dataframe['it_sim_dis_diff_test'] > 0, 'Tested', 'Untested')
/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_48599/2665625952.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe['Distractor V2 Similarity Preference Tested'] = dataframe['v2_prefers_test']
/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_48599/2665625952.py:36: Perfo

In [151]:
df['it_sim_dis_diff']

19113   -0.606126
19114   -0.477866
19115    0.131423
19116    0.127326
19117   -0.477398
           ...   
30808   -0.130615
30809    0.362738
30810    1.088831
30811    0.844930
30812    1.094318
Name: it_sim_dis_diff, Length: 42000, dtype: float64

In [152]:
# md = smf.glm(
#     "resp_correct ~ it_sim_dis_diff*reliability*validity \
#         + v2_sim_dis_diff*reliability*validity",
#     data=df,
#     family=sm.families.Binomial()  
# ).fit()

# # View the summary
# md.summary()

In [153]:
df['reliability_binary'] = (df['reliability'] > 0.7).astype(int)
df['reliability_binary']

/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_48599/2146888266.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['reliability_binary'] = (df['reliability'] > 0.7).astype(int)


19113    1
19114    0
19115    0
19116    1
19117    1
        ..
30808    1
30809    1
30810    0
30811    1
30812    1
Name: reliability_binary, Length: 42000, dtype: int64

In [154]:
df['validity_binary'] = (df['validity'] == 'valid').astype(int)
df['validity_binary']

/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_48599/324637376.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['validity_binary'] = (df['validity'] == 'valid').astype(int)


19113    1
19114    0
19115    0
19116    1
19117    1
        ..
30808    0
30809    1
30810    1
30811    1
30812    1
Name: validity_binary, Length: 42000, dtype: int64

In [155]:
# # Convert binary variables to categorical data types
# df['reliability_binary'] = df['reliability_binary'].astype('category')
# df['validity_binary'] = df['validity_binary'].astype('category')

In [156]:
df = df.loc[:, ~df.columns.duplicated()]

# Convert binary variables to categorical data types
# df['reliability_binary'] = df['reliability'].astype('category')
# df['validity_binary'] = df['validity'].astype('category')

# Fit the generalized linear model (GLM) with binomial family
md = smf.glm(
    "Accuracy ~ it_sim_dis_diff * reliability_binary * validity_binary + v2_sim_dis_diff * reliability_binary * validity_binary",
    data=df,
    family=sm.families.Binomial()
).fit()

# Display the model summary
print(md.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Accuracy   No. Observations:                42000
Model:                            GLM   Df Residuals:                    41988
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -23298.
Date:                Mon, 10 Feb 2025   Deviance:                   1.3910e+05
Time:                        16:15:28   Pearson chi2:                 3.88e+04
No. Iterations:                     5   Pseudo R-squ. (CS):            0.01033
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [157]:
df['Prioritized - Deprioritized IT Distractor Similarity']

19113                
19114                
19115                
19116                
19117                
             ...     
30808                
30809                
30810    Most Similar
30811    Most Similar
30812    Most Similar
Name: Prioritized - Deprioritized IT Distractor Similarity, Length: 42000, dtype: category
Categories (5, object): ['Least Similar' < '' < ' ' < '   ' < 'Most Similar']

In [158]:
df['reliability_binary']

19113    1
19114    0
19115    0
19116    1
19117    1
        ..
30808    1
30809    1
30810    0
30811    1
30812    1
Name: reliability_binary, Length: 42000, dtype: int64

In [159]:
# df['Prioritized - Deprioritized IT Distractor Similarity'] = pd.Categorical(
#     df['Prioritized - Deprioritized IT Distractor Similarity'],
#     categories=['Least Similar', '', ' ', '   ', 'Most Similar'],
#     ordered=True
# )

# df['Prioritized - Deprioritized V2 Distractor Similarity'] = pd.Categorical(
#     df['Prioritized - Deprioritized V2 Distractor Similarity'],
#     categories=['Least Similar', '', ' ', '   ', 'Most Similar'],
#     ordered=True
# )
# df['V2 Distractor Similarity\nto Prioritized Item'] = pd.Categorical(
#     df['V2 Distractor Similarity\nto Prioritized Item'],
#     categories=['Least Similar', '', ' ', '   ', 'Most Similar'],
#     ordered=True
# )
# df['IT Distractor Similarity\nto Prioritized Item'] = pd.Categorical(
#     df['IT Distractor Similarity\nto Prioritized Item'],
#     categories=['Least Similar', '', ' ', '   ', 'Most Similar'],
#     ordered=True
# )
# df['V2 Distractor Similarity\nto Derioritized Item'] = pd.Categorical(
#     df['Prioritized - Deprioritized V2 Distractor Similarity'],
#     categories=['Least Similar', '', ' ', '   ', 'Most Similar'],
#     ordered=True
# )
# df['IT Distractor Similarity\nto Derioritized Item'] = pd.Categorical(
#     df['Prioritized - Deprioritized V2 Distractor Similarity'],
#     categories=['Least Similar', '', ' ', '   ', 'Most Similar'],
#     ordered=True
# )

In [160]:
# df.to_csv('pilot6_nonresp_inc.csv', index=False)


In [161]:
df = df[df['taskPhase'] == 'mainTask']


In [162]:
len(set(df['participant']))

137

In [163]:
set(df['participant'])

{123985,
 124081,
 124180,
 124750,
 124819,
 125722,
 128407,
 129019,
 130426,
 130429,
 130483,
 130498,
 131011,
 132901,
 133036,
 133162,
 133999,
 135229,
 135403,
 135787,
 136771,
 138529,
 138907,
 139135,
 140563,
 141181,
 141394,
 141703,
 141739,
 142204,
 142780,
 143425,
 143905,
 144541,
 145489,
 145672,
 145915,
 147487,
 147691,
 148036,
 148378,
 148486,
 149635,
 149656,
 150043,
 150058,
 151222,
 151915,
 151981,
 152644,
 152713,
 152719,
 152953,
 153394,
 153442,
 153454,
 154006,
 154240,
 154567,
 154723,
 155275,
 155866,
 156313,
 156430,
 157615,
 157828,
 158026,
 158164,
 158854,
 159262,
 159346,
 159529,
 160456,
 160768,
 161827,
 162313,
 162328,
 162373,
 164524,
 164581,
 164596,
 165064,
 165337,
 165355,
 165586,
 165658,
 165835,
 165862,
 166552,
 166588,
 166894,
 166972,
 167176,
 167353,
 167389,
 167407,
 167431,
 167482,
 167863,
 167869,
 167875,
 167917,
 167956,
 168001,
 168010,
 168034,
 168073,
 168085,
 168412,
 168415,
 168493,
 

In [164]:

nan_values = df['V2 Distractor Similarity\nto Prioritized Item'].isna()
rows_with_nan = df[nan_values]
print(rows_with_nan)


Empty DataFrame
Columns: [setup_js.started, setup_js.stopped, participant, date, expName, psychopyVersion, OS, frameRate, Instructions1.started, Instructions1.stopped, key_resp_2.keys, key_resp_2.rt, key_resp_2.duration, Instructions2.started, Instructions2.stopped, key_resp_3.keys, key_resp_3.rt, key_resp_3.duration, Instructions3.started, Instructions3.stopped, key_resp_4.keys, key_resp_4.rt, key_resp_4.duration, Instructions4.started, Instructions4.stopped, key_resp_5.keys, key_resp_5.rt, key_resp_5.duration, Instructions5.started, Instructions5.stopped, key_resp_10.keys, key_resp_10.rt, key_resp_10.duration, Instructions6b.started, Instructions6b.stopped, key_resp_16.keys, key_resp_16.rt, key_resp_16.duration, Instructions6.started, Instructions6.stopped, key_resp_13.keys, key_resp_13.rt, key_resp_13.duration, memoranda.started, memoranda.stopped, isi1.started, isi1.stopped, probe.started, probe.stopped, mouse.x, mouse.y, mouse.leftButton, mouse.midButton, mouse.rightButton, mouse.

In [165]:
df

,setup_js.started,setup_js.stopped,participant,date,expName,psychopyVersion,OS,frameRate,Instructions1.started,Instructions1.stopped,...,it_sim_dis_unattend_sq,v2_sim_dis_unattend_sq,it_sim_dis_diff_test_sq,v2_sim_dis_diff_test_sq,v2_sim_dis_test_sq,it_sim_dis_test_sq,it_sim_dis_untest_sq,v2_sim_dis_untest_sq,reliability_binary,validity_binary
19113,NaN,NaN,123985,2025-01-28 21:25:50.434,WM_Deepgen,2023.2.1,MacIntel,61.349693,NaN,NaN,...,0.478914,0.005223,0.367389,0.000300,0.003021,0.007380,0.478914,0.005223,1,1
19114,NaN,NaN,123985,2025-01-28 21:25:50.434,WM_Deepgen,2023.2.1,MacIntel,61.349693,NaN,NaN,...,0.052110,0.000007,0.228356,0.025121,0.000007,0.052110,0.062295,0.025964,0,0
19115,NaN,NaN,123985,2025-01-28 21:25:50.434,WM_Deepgen,2023.2.1,MacIntel,61.349693,NaN,NaN,...,0.000245,0.000005,0.017272,0.016261,0.000005,0.000245,0.021634,0.016850,0,0
19116,NaN,NaN,123985,2025-01-28 21:25:50.434,WM_Deepgen,2023.2.1,MacIntel,61.349693,NaN,NaN,...,0.000247,0.004356,0.016212,0.000809,0.001410,0.012454,0.000247,0.004356,1,1
19117,NaN,NaN,123985,2025-01-28 21:25:50.434,WM_Deepgen,2023.2.1,MacIntel,61.349693,NaN,NaN,...,0.218285,0.001367,0.227909,0.001513,0.005757,0.000104,0.218285,0.001367,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30808,NaN,NaN,171304,2025-01-29 19:25:41.828,WM_Deepgen,2023.2.1,MacIntel,59.880240,NaN,NaN,...,0.053325,0.000691,0.017060,0.005176,0.000691,0.053325,0.010061,0.009650,1,0
30809,NaN,NaN,171304,2025-01-29 19:25:41.828,WM_Deepgen,2023.2.1,MacIntel,59.880240,NaN,NaN,...,0.032701,0.020746,0.131579,0.014833,0.000495,0.295471,0.032701,0.020746,1,1
30810,NaN,NaN,171304,2025-01-29 19:25:41.828,WM_Deepgen,2023.2.1,MacIntel,59.880240,NaN,NaN,...,0.086701,0.000924,1.185553,0.009563,0.016433,0.631041,0.086701,0.000924,0,1
30811,NaN,NaN,171304,2025-01-29 19:25:41.828,WM_Deepgen,2023.2.1,MacIntel,59.880240,NaN,NaN,...,0.055322,0.014797,0.713906,0.005664,0.002151,0.371764,0.055322,0.014797,1,1


In [166]:
df_nonresp = clean_dataframe_from_nonresp(df)


In [167]:
# df_nonresp.to_csv('pilot6_shuffled_nonresp_exc.csv', index=False)


In [168]:
len(set(df_nonresp['participant']))

137

In [169]:
len((df_nonresp['participant']))

40039

In [170]:
participant_counts = df_nonresp['participant'].value_counts()
participant_counts_df = participant_counts.reset_index()
participant_counts_df.columns = ['participant', 'count']

print(min(participant_counts_df['count']))

201


In [171]:
# df_old = pd.read_csv('./pilot4_shuffled_nonresp_inc.csv')
# numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns


In [172]:
# len(set(df_old['participant']))

In [173]:
from sklearn.preprocessing import StandardScaler


In [174]:

df['it_sim_dis_diff'] = df['it_sim_dis_diff'] - df['it_sim_dis_diff'].mean(skipna=True)
df['it_sim_dis_diff_sq'] = df['it_sim_dis_diff_sq'] - df['it_sim_dis_diff_sq'].mean(skipna=True)
df['v2_sim_dis_diff'] = df['v2_sim_dis_diff'] - df['v2_sim_dis_diff'].mean(skipna=True)
df['v2_sim_dis_diff_sq'] = df['v2_sim_dis_diff_sq'] - df['v2_sim_dis_diff_sq'].mean(skipna=True)

df['it_sim_dis_diff_test'] = df['it_sim_dis_diff_test'] - df['it_sim_dis_diff_test'].mean(skipna=True)
df['it_sim_dis_diff_test_sq'] = df['it_sim_dis_diff_test_sq'] - df['it_sim_dis_diff_test_sq'].mean(skipna=True)
df['v2_sim_dis_diff_test'] = df['v2_sim_dis_diff_test'] - df['v2_sim_dis_diff_test'].mean(skipna=True)
df['v2_sim_dis_diff_test_sq'] = df['v2_sim_dis_diff_test_sq'] - df['v2_sim_dis_diff_test_sq'].mean(skipna=True)


df['it_sim_dis_attend'] = df['it_sim_dis_attend'] - df['it_sim_dis_attend'].mean(skipna=True)
df['v2_sim_dis_attend'] = df['v2_sim_dis_attend'] - df['v2_sim_dis_attend'].mean(skipna=True)
df['it_sim_dis_unattend'] = df['it_sim_dis_unattend'] - df['it_sim_dis_unattend'].mean(skipna=True)
df['v2_sim_dis_unattend'] = df['v2_sim_dis_unattend'] - df['v2_sim_dis_unattend'].mean(skipna=True)

df['it_sim_dis_test'] = df['it_sim_dis_test'] - df['it_sim_dis_test'].mean(skipna=True)
df['v2_sim_dis_test'] = df['v2_sim_dis_test'] - df['v2_sim_dis_test'].mean(skipna=True)
df['it_sim_dis_untest'] = df['it_sim_dis_untest'] - df['it_sim_dis_untest'].mean(skipna=True)
df['v2_sim_dis_untest'] = df['v2_sim_dis_untest'] - df['v2_sim_dis_untest'].mean(skipna=True)



# Standardize (z-score) each column
scaler = StandardScaler()
df['v2_sim_dis_diff_z'] = scaler.fit_transform(df[['v2_sim_dis_diff']])
df['it_sim_dis_diff_z'] = scaler.fit_transform(df[['it_sim_dis_diff']])
df['v2_sim_dis_diff_sq_z'] = scaler.fit_transform(df[['v2_sim_dis_diff_sq']])
df['it_sim_dis_diff_sq_z'] = scaler.fit_transform(df[['it_sim_dis_diff_sq']])

df['v2_sim_dis_diff_test_z'] = scaler.fit_transform(df[['v2_sim_dis_diff_test']])
df['it_sim_dis_diff_test_z'] = scaler.fit_transform(df[['it_sim_dis_diff_test']])
df['v2_sim_dis_diff_test_sq_z'] = scaler.fit_transform(df[['v2_sim_dis_diff_test_sq']])
df['it_sim_dis_diff_test_sq_z'] = scaler.fit_transform(df[['it_sim_dis_diff_test_sq']])


df['it_sim_dis_attend_z'] = scaler.fit_transform(df[['it_sim_dis_attend']])
df['v2_sim_dis_attend_z'] = scaler.fit_transform(df[['v2_sim_dis_attend']])
df['it_sim_dis_unattend_z'] = scaler.fit_transform(df[['it_sim_dis_unattend']])
df['v2_sim_dis_unattend_z'] = scaler.fit_transform(df[['v2_sim_dis_unattend']])

df['it_sim_dis_test_z'] = scaler.fit_transform(df[['it_sim_dis_test']])
df['v2_sim_dis_test_z'] = scaler.fit_transform(df[['v2_sim_dis_test']])
df['it_sim_dis_untest_z'] = scaler.fit_transform(df[['it_sim_dis_untest']])
df['v2_sim_dis_untest_z'] = scaler.fit_transform(df[['v2_sim_dis_untest']])

# # Convert specified columns to categorical (factor-like) data types
# df['reliability_binary'] = df['reliability_binary'].astype('category')
# df['validity_binary'] = df['validity_binary'].astype('category')

df['reliability_binary'] = df['reliability_binary'].astype('category')
df['validity_binary'] = df['validity_binary'].astype('category')

df['reliability_binary'] = np.where(df['reliability_binary'] == 0, -1 , 1)
df['validity_binary'] = np.where(df['validity_binary'] == 0, -1 , 1)

# df['validity_binary'] 
df.to_csv('pilot6_nonresp_inc_test_diff.csv', index=False)


In [175]:
# df['it_sim_dis_diff'] = df['it_sim_dis_diff'] - np.mean(df['it_sim_dis_diff'])
# df['v2_sim_dis_diff'] = df['v2_sim_dis_diff'] - np.mean(df['v2_sim_dis_diff'])







# df['it_sim_dis_test'] = df['it_sim_dis_test'] - np.mean(df['it_sim_dis_test'])
# df['v2_sim_dis_test'] = df['v2_sim_dis_test'] - np.mean(df['v2_sim_dis_test'])

# df['it_sim_dis_diff_test'] = df['it_sim_dis_diff_test'] - np.mean(df['it_sim_dis_diff_test'])
# df['v2_sim_dis_diff_test'] = df['v2_sim_dis_diff_test'] - np.mean(df['v2_sim_dis_diff_test'])





# df['it_sim_dis_diff_sq'] = [x**2 for x in df['it_sim_dis_diff']]
# df['v2_sim_dis_diff_sq'] = [x**2 for x in df['v2_sim_dis_diff']]





# df['v2_sim_dis_test_sq'] = [x**2 for x in df['v2_sim_dis_test']]
# df['it_sim_dis_test_sq'] = [x**2 for x in df['it_sim_dis_test']]

# df['it_sim_dis_diff_test_sq'] = [x**2 for x in df['it_sim_dis_diff_test']]
# df['v2_sim_dis_diff_test_sq'] = [x**2 for x in df['v2_sim_dis_diff_test']]





# df['it_sim_dis_diff_sq'] = df['it_sim_dis_diff_sq'] - np.mean(df['it_sim_dis_diff_sq'])
# df['v2_sim_dis_diff_sq'] = df['v2_sim_dis_diff_sq'] - np.mean(df['v2_sim_dis_diff_sq'])



# df['v2_sim_dis_test_sq'] = df['v2_sim_dis_test_sq'] - np.mean(df['v2_sim_dis_test_sq'])
# df['it_sim_dis_test_sq'] = df['it_sim_dis_test_sq'] - np.mean(df['it_sim_dis_test_sq'])

# df['it_sim_dis_diff_test_sq'] = df['it_sim_dis_diff_test_sq'] - np.mean(df['it_sim_dis_diff_test_sq'])
# df['v2_sim_dis_diff_test_sq'] = df['v2_sim_dis_diff_test_sq'] - np.mean(df['v2_sim_dis_diff_test_sq'])




# df['IT_diff_binned'] = pd.qcut(df['it_sim_dis_diff'], 5, duplicates='drop')
# df['V2_diff_binned'] = pd.qcut(df['v2_sim_dis_diff'], 5, duplicates='drop')
# df['IT_diff_binned_sq'] = pd.qcut(df['it_sim_dis_diff_sq'], 5, duplicates='drop')
# df['V2_diff_binned_sq'] = pd.qcut(df['v2_sim_dis_diff_sq'], 5, duplicates='drop')




# df['IT_diff_binned_test'] = pd.qcut(df['it_sim_dis_diff_test'], 5, duplicates='drop')
# df['V2_diff_binned_test'] = pd.qcut(df['v2_sim_dis_diff_test'], 5, duplicates='drop')
# df['IT_diff_binned_test_sq'] = pd.qcut(df['it_sim_dis_diff_test_sq'], 5, duplicates='drop')
# df['V2_diff_binned_test_sq'] = pd.qcut(df['v2_sim_dis_diff_test_sq'], 5, duplicates='drop')

# def round_to_significant_figures(x, sig_figs=2):
#     if x == 0:
#         return 0
#     else:
#         return round(x, sig_figs - int(np.floor(np.log10(abs(x)))) - 1)


# def process_interval(interval):
#     # Round both bounds
#     lower_rounded = round_to_significant_figures(interval.left)
#     upper_rounded = round_to_significant_figures(interval.right)
#     # Return a new interval with the rounded bounds
#     return pd.Interval(lower_rounded, upper_rounded)

# df['IT_diff_binned'] = df['IT_diff_binned'].apply(process_interval)
# df['V2_diff_binned'] = df['V2_diff_binned'].apply(process_interval)
# df['IT_diff_binned_sq'] = df['IT_diff_binned_sq'].apply(process_interval)
# df['V2_diff_binned_sq'] = df['V2_diff_binned_sq'].apply(process_interval)

# df['IT_diff_binned_test'] = df['IT_diff_binned_test'].apply(process_interval)
# df['V2_diff_binned_test'] = df['V2_diff_binned_test'].apply(process_interval)
# df['IT_diff_binned_test_sq'] = df['IT_diff_binned_test_sq'].apply(process_interval)
# df['V2_diff_binned_test_sq'] = df['V2_diff_binned_test_sq'].apply(process_interval)

# df['Prioritized - Deprioritized V2 Distractor Similarity Ranges'] = df['V2_diff_binned']
# df['Prioritized - Deprioritized IT Distractor Similarity Ranges'] = df['IT_diff_binned']
# df['Prioritized - Deprioritized V2 Distractor Similarity Squared Ranges'] = df['V2_diff_binned_sq']
# df['Prioritized - Deprioritized IT Distractor Similarity Squared Ranges'] = df['IT_diff_binned_sq']

# df['Tested - Untested V2 Distractor Similarity Ranges'] = df['V2_diff_binned_test']
# df['Tested - Untested IT Distractor Similarity Ranges'] = df['IT_diff_binned_test']
# df['Tested - Untested V2 Distractor Similarity Squared Ranges'] = df['V2_diff_binned_test_sq']
# df['Tested - Untested IT Distractor Similarity Squared Ranges'] = df['IT_diff_binned_test_sq']

In [176]:
df['reliability_binary'] 

19113    1
19114   -1
19115   -1
19116    1
19117    1
        ..
30808    1
30809    1
30810   -1
30811    1
30812    1
Name: reliability_binary, Length: 42000, dtype: int64

In [177]:
len(set(df['participant']))


137

In [178]:
df['reliability_binary']

19113    1
19114   -1
19115   -1
19116    1
19117    1
        ..
30808    1
30809    1
30810   -1
30811    1
30812    1
Name: reliability_binary, Length: 42000, dtype: int64

In [179]:
# df = pd.read_csv('pilot5_shuffled_nonresp_inc_stats.csv')


In [180]:
scaler = StandardScaler()




df['it_sim_dis_attend_sq_z'] = scaler.fit_transform(df[['it_sim_dis_attend_sq']])
df['it_sim_dis_unattend_sq_z'] = scaler.fit_transform(df[['it_sim_dis_unattend_sq']])
df['v2_sim_dis_attend_sq_z'] = scaler.fit_transform(df[['v2_sim_dis_attend_sq']])
df['v2_sim_dis_unattend_sq_z'] = scaler.fit_transform(df[['v2_sim_dis_unattend_sq']])


In [181]:
df.to_csv('pilot6_nonresp_inc_test_diff_stats.csv', index=False)


In [182]:
print('hello')

hello
